## ISAAA crop genomics webscrape

**Description**: 

**Usage**: 

**Notes**: 

Last Edited: 

Author: LNC

Contact: lisa.cao@cybera.ca

In [40]:
#import libraries
import requests 
import pandas as pd
from bs4 import BeautifulSoup 
import csv 
import lxml

In [41]:
# get raw html
URL = "https://www.isaaa.org/gmapprovaldatabase/geneslist/default.asp"
gmo = requests.get(URL) 
soup = BeautifulSoup(gmo.content, "lxml")
# get text 
souptext = str(soup.get_text())

In [42]:
# get rid of leading and trailing whitespace
genetext_raw = souptext.splitlines() 

# remove leading and trailing whitespace
genetext = []
for element in genetext_raw: 
    genetext_entry = element.strip()
    genetext.append(genetext_entry)

# remove empty list entries 
while('' in genetext) : 
    genetext.remove('') 

# slice out only parts we want
genetext = genetext[34:-15]

In [43]:
traits = [info for info in genetext if "Trait:" in info] # get list of traits

In [44]:
sublist = [] #generate list  of lists grouped by trait 
traitinfo = []
for info in genetext: 
    if info in traits: 
        if sublist:
            traitinfo.append(sublist)
        sublist = [info]
    else: 
        sublist.append(info)
traitinfo.append(sublist)

In [45]:
### DIRTY FIX FOR dCS
traitinfo[23][5:6] = [''.join(traitinfo[23][5:7])] #append product info that was split 
del traitinfo[23][6]
traitinfo[23]

['Trait: Low Gossypol',
 'dCS',
 'Source:',
 'Gossypium hirsutum L.',
 'Product:',
 'dsRNA that suppresses the expression of d-cadinene synthase gene that encode d-cadinene synthase, a key enzyme involved in gossypolbiosynthesis, thru RNAi pathway',
 'Function:',
 'silence the endogenous dCS genes']

In [46]:
#convert to df and drop first row
df = pd.DataFrame(traitinfo)
df = pd.DataFrame.dropna(df, how = 'all')
df = df.drop(0,0)
df

,0,1,2,3,4,5,6,7,8,9,...,172,173,174,175,176,177,178,179,180,181
1,"Trait: 2,4-D herbicide tolerance",aad-1,Source:,synthetic form of the aad-1 gene from Sphingob...,Product:,aryloxyalkanoate dioxygenase 1 (AAD-1) protein,Function:,"detoxifies 2,4-D herbicide by side-chain degra...",aad-12,Source:,...,None,None,None,None,None,None,None,None,None,None
2,Trait: Altered lignin production,ccomt (inverted repeat),Source:,Medicago sativa (alfalfa),Product:,dsRNA that suppresses endogenous S-adenosyl-L-...,Function:,reduces content of guaiacyl (G) lignin,EgCAld5H,Source:,...,None,None,None,None,None,None,None,None,None,None
3,Trait: Anti-allergy,7crp,Source:,synthetic form of tolerogenic protein from Cry...,Product:,modified cry j 1 and cry j 2 pollen antigens c...,Function:,triggers mucosal immune tolerance to cedar pol...,None,None,...,None,None,None,None,None,None,None,None,None,None
4,Trait: Antibiotic resistance,aad,Source:,Escherichia coli,Product:,3''(9)-O-aminoglycoside adenylyltransferase en...,Function:,allows selection for resistance to aminoglycos...,aph4 (hpt),Source:,...,None,None,None,None,None,None,None,None,None,None
5,Trait: Coleopteran insect resistance,cry34Ab1,Source:,Bacillus thuringiensis strain PS149B1,Product:,Cry34Ab1 delta-endotoxin,Function:,confers resistance to coleopteran insects part...,cry35Ab1,Source:,...,None,None,None,None,None,None,None,None,None,None
6,Trait: Delayed fruit softening,pg (sense or antisense),Source:,Lycopersicon esculentum,Product:,no functional polygalacturonase enzyme is prod...,Function:,inhibits the production of polygalacturonase e...,None,None,...,None,None,None,None,None,None,None,None,None,None
7,Trait: Delayed ripening/senescence,acc,Source:,Ananas comosus,Product:,1-aminocyclopropane-1-carboxylic acid synthase,Function:,involved in catalyzing the penultimate step in...,acc (truncated),Source:,...,None,None,None,None,None,None,None,None,None,None
8,Trait: Dicamba herbicide tolerance,dmo,Source:,Stenotrophomonas maltophilia strain DI-6,Product:,dicamba mono-oxygenase enzyme,Function:,confers tolerance to the herbicide dicamba (2-...,None,None,...,None,None,None,None,None,None,None,None,None,None
9,Trait: Drought stress tolerance,cspB,Source:,Bacillus subtilis,Product:,cold shock protein B,Function:,maintains normal cellular functions under wate...,EcBetA,Source:,...,None,None,None,None,None,None,None,None,None,None
10,Trait: Enhanced Photosynthesis/Yield,bbx32,Source:,Arabidopsis thaliana,Product:,protein that interacts with one or more endoge...,Function:,to modulate plant's diurnal biology and to enh...,None,None,...,None,None,None,None,None,None,None,None,None,None


In [52]:
data_full = [] ### main list in which we will populate rows
for index, row in df.iterrows(): ### you can use df.iterrows() here
    count = 1  ### Initiating the count
    ### Now we want to run a while loop for each row until it encounters None
    while (row[count] is not None) & (count < df.shape[1]):
        
        data_part = [] ### we will be adding this list to all_rows list defined earlier
        
        keywords = ["Source:", "Function:", "Product:"] #error handling for empty function:
        
        if row[count] in keywords: #DIRTY FIX
            count = count - 1 
            
        data_part.append(row[0]) #trait 
        data_part.append(row[count]) # gene
        data_part.append(row[count + 2]) # source
        data_part.append(row[count + 4]) # product
        data_part.append(row[count + 6]) # function

        data_full.append(data_part) # append to main list
        
        count = count + 7 ### Because each gene occupies 7 columns
        
        # break statement 
        if count >= df.shape[1]:
            break
        
### Finally, we wanna use main list to convert to dataframe
traitsdata = pd.DataFrame(data_full) 

In [54]:
traitsdata[0] = traitsdata[0].map(lambda x: x.lstrip('Trait: ')) # strip text from trait column
traitsdata.columns = ['Trait', "Gene","Product", "Source", "Function"] #rename columns
traitsdata = traitsdata[1:]

In [58]:
#export to csv 
traitsdata.to_csv('isaaa_croptraits.csv', index = False)

In [64]:
traitsdata[5:50]

,Trait,Gene,Product,Source,Function
6,Antibiotic resistance,aph4 (hpt),Escherichia coli,hygromycin-B phosphotransferase (hph) enzyme,allows selection for resistance to the antibio...
7,Antibiotic resistance,bla,Escherichia coli,beta lactamase enzyme,detoxifies beta lactam antibiotics such as amp...
8,Antibiotic resistance,hph,Streptomyces sp.,hygromycin phosphotransferase,allows selection for resistance to the antibio...
9,Antibiotic resistance,nptII,Escherichia coli Tn5 transposon,neomycin phosphotransferase II enzyme,allows transformed plants to metabolize neomyc...
10,Antibiotic resistance,spc,Escherichia coli,spectinomycin adenyl transferase enzyme (not e...,confers resistance to spectinomycin/streptomyc...
11,Coleopteran insect resistance,cry34Ab1,Bacillus thuringiensis strain PS149B1,Cry34Ab1 delta-endotoxin,confers resistance to coleopteran insects part...
12,Coleopteran insect resistance,cry35Ab1,Bacillus thuringiensis strain PS149B1,Cry35Ab1 delta-endotoxin,confers resistance to coleopteran insects part...
13,Coleopteran insect resistance,cry3A,Bacillus thuringiensis subsp. tenebrionis,cry3A delta endotoxin,confers resistance to coleopteran insects by s...
14,Coleopteran insect resistance,cry3Bb1,Bacillus thuringiensis subsp. kumamotoensis,Cry3Bb1 delta endotoxin,confers resistance to coleopteran insects part...
15,Coleopteran insect resistance,dvsnf7,Western Corn Rootworm (Diabrotica virgifera vi...,double-stranded RNA transcript containing a 24...,RNAi interference resulting to down-regulation...


In [61]:
x = traitsdata['Gene'].unique()

In [63]:
print(sorted(x))

['2mepsps', '5AT', '7crp', 'API', 'AtAHAS', 'CpTI', 'EcBetA', 'EgCAld5H', 'Hahb-4', 'Lackl-delta12D', 'Micpu-delta-6D', 'Nc.Fad3', 'NtQPT1 (antisense)', 'OsGA2ox1', 'OtD5E', 'OtD6D', 'PGAS PPO suppression gene', 'Pavsa-delta-4D', 'Pavsa-delta-5D', 'PhL', 'PiO3D', 'Picpa-omega-3D', 'PirO3D', 'Pj.D6D', 'PlD4D', 'PpD6E', 'PsD12D', 'Psy', 'Pyrco-delta-5E', 'Pyrco-delta-6E', 'R1', 'RB', 'RmBetA', 'Rpi-vnt1', 'S4-HrA', 'TcD4D', 'TcD5D', 'TpD6E', 'Vlnv', 'aad', 'aad-12', 'ac1 (sense and antisense)', 'acc', 'acc (truncated)', 'accd', 'als', 'amy797E', 'anti-efe', 'aph4 (hpt)', 'asn1', 'athb17', 'avhppd-03', 'b-Lyc', 'bar', 'barnase', 'barstar', 'bbx32', 'bla', "bp40 (f3'5'h)", 'bxn', 'ccomt (inverted repeat)', 'cel1', 'cmv_cp', 'cordapA', 'cp4 epsps (aroA:CP4)', 'crt1', 'cry1A', 'cry1A.105', 'cry1Ab', 'cry1Ab (truncated)', 'cry1Ab-Ac', 'cry1Ac', 'cry1C', 'cry1F', 'cry1Fa2', 'cry2Ab2', 'cry2Ae', 'cry34Ab1', 'cry35Ab1', 'cry3A', 'cry3Bb1', 'cry9C', 'cspB', 'csr1-2', 'cytb5', 'dCS', 'dam', 'dfr',

In [2]:
from bio import SeqIO

ModuleNotFoundError: No module named 'bio'

In [1]:
from Bio.Seq import Seq

ModuleNotFoundError: No module named 'Bio'

In [2]:
import Bio

ModuleNotFoundError: No module named 'Bio'

In [ ]:
use str(fasta.seq) instead
